# $ West Nile Project 4 $
### $ Predicting West Nile in Chicago $
#### Contributors: Will Suh, Ahbishek Sharma, Uday Datta, Jon Lau

In [ ]:
!pip install vincenty #measures distance between lat and long

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns

from vincenty import vincenty 
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

import time
from datetime import datetime
%autocall 1

# Importing Datasets




In [ ]:
spray = pd.read_csv("./data/west_nile/input/spray.csv")
spray.head()

In [ ]:
#weather_file = files.upload()
weather = pd.read_csv('./data/west_nile/input/weather.csv')
weather.head()

In [ ]:
#train_file = files.upload()
train = pd.read_csv('./data/west_nile/input/train.csv')
train.head()

In [ ]:
#import Test File
#test_file = files.upload()
test = pd.read_csv('./data/west_nile/input/test.csv')
test.head()

# EDA

In [ ]:
def EDA(df):
    null_vals = df.isnull().sum()[df.isnull().sum() > 0] 
    shape = df.shape
    dtypes = df.dtypes
    print('Nulls:', null_vals)
    print('Shape:', shape)
    print('Data Types:', dtypes)

In [ ]:
print(EDA(train))
print(EDA(test))
print(EDA(spray))
print(EDA(weather))
#print(EDA(sample))

 No np.NaNs in our data. Nulls are represented with M in some fields. 

In [ ]:
# date is an object an int
def convert_date(df):
    df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
convert_date(train)
convert_date(test)
convert_date(weather)

In [ ]:
#plotted mosquitos by trap by date
train[['Date', 'Trap', 'NumMosquitos']].groupby(by = ['Date','Trap'])['Date','Trap','NumMosquitos'] \
    .sum().reset_index().sort_values('NumMosquitos', ascending = False).set_index('Date').plot(style = '.')
    
plt.title('Mosquitos by Trap over Time')
plt.xlabel('Year')
plt.ylabel('Mosquitos at Traps');

In [ ]:
#plotted WNV incidents by trap
train[['Date', 'Trap','WnvPresent']].groupby(by = ['Date','Trap'])['Date','Trap','WnvPresent'] \
    .sum().reset_index().sort_values('WnvPresent', ascending = False).set_index('Date').plot(style = '.')
    
plt.title('Positive WnV Cases by Trap over Time')
plt.xlabel('Year')
plt.ylabel('WnV Present at Traps');

Allowing us to visually undestand out timeseries data. 

In [ ]:
#Create new column for combined Lat and Long
train['LatLong'] = list(zip(train.Latitude, train.Longitude))
test['LatLong'] = list(zip(test['Latitude'], test['Longitude']))

station1 = (41.995, -87.933)
station2 = (41.786, -87.752)
train['Closest_Station'] = [ 1 if vincenty(x,station1) < vincenty(x,station2) else 2 for x in train['LatLong']]
test['Closest_Station'] = [ 1 if vincenty(x,station1) < vincenty(x,station2) else 2 for x in test['LatLong']]

In [ ]:
#Merge DataFramne

train_weather = train.merge(weather,how = 'left', left_on = ['Date','Closest_Station'],right_on =['Date','Station'])
test_weather = test.merge(weather,how = 'left', left_on = ['Date','Closest_Station'],right_on =['Date','Station'])

In [ ]:
train_weather = train_weather.drop(columns = ['SeaLevel','CodeSum', 'Sunrise', 'Sunset','Depart','Depth','Water1', 'SnowFall', 'Cool', 'Heat','StnPressure', 'AvgSpeed','ResultSpeed','ResultDir','NumMosquitos'])
test_weather =test_weather.drop(columns = ['SeaLevel','CodeSum', 'Sunrise', 'Sunset','Depart','Depth','Water1', 'SnowFall', 'Cool', 'Heat','StnPressure', 'AvgSpeed','ResultSpeed','ResultDir'])

Dropping columns that we don't se as useful. 

In [ ]:
#filling in missing(M) and trace(T)
weather_dataset = train_weather.columns.tolist()

for col in weather_dataset:
    for row in range(train_weather.shape[0]):
        if train_weather.loc[row, col] == 'M' or train_weather.loc[row, col] == '  T':
            train_weather.loc[row, col] = train_weather.loc[row - 1, col]

In [ ]:
weather_dataset2 = test_weather.columns.tolist()

for col in weather_dataset2:
    for row in range(test_weather.shape[0]):
        if test_weather.loc[row, col] == 'M' or test_weather.loc[row, col] == '  T':
            test_weather.loc[row, col] = test_weather.loc[row - 1, col]

This fills in the T and M data with closest data. Time series allows us to do this knowing that the data is corrlated. 
This allows the data to be all numerical for modeling. (same for cell below)

In [ ]:

weather_object_dtypes = ['Tavg', 'WetBulb', 'PrecipTotal']

for col in weather_object_dtypes:
    train_weather[col] = pd.to_numeric(train_weather[col])
    test_weather[col] = pd.to_numeric(test_weather[col])

In [ ]:
def mosquito(df):
    #split wnv transmitting mosquito species lines into separate columns
    df['CULEX PIPIENS'] = 0
    df['CULEX RESTUANS'] = 0
    
    for row in range(df.shape[0]):
        if df.loc[row, 'Species'] == 'CULEX PIPIENS/RESTUANS':
            df.loc[row, 'CULEX PIPIENS'] == 1
            df.loc[row, 'CULEX RESTUANS'] == 1
        elif df.loc[row, 'Species'] == 'CULEX PIPIENS':
            df.loc[row, 'CULEX PIPIENS'] == 1
        elif df.loc[row, 'Species'] == 'CULEX RESTUANS':
            df.loc[row, 'CULEX RESTUANS'] == 1
            
    
    df.drop(columns = ['Species'], inplace = True)

In [ ]:
#call function
mosquito(train_weather)
mosquito(test_weather)

In [ ]:
train_weather = pd.get_dummies(train_weather, columns = ['Block', 'Trap'])
test_weather = pd.get_dummies(test_weather, columns = ['Block', 'Trap'])

Turning the categorical data into numerical data with function and get dummeies to prepare for modeling. 

In [ ]:
# interaction features
train_weather['wet_temp'] = train_weather['PrecipTotal']*train_weather['Tavg']
train_weather['wet_temp_roll'] = train_weather['wet_temp'].rolling(3).mean()
train_weather['wet_temp_roll'].fillna(0, inplace = True)
train_weather['wet_temp_roll_shift14'] = train_weather['wet_temp_roll'].shift(14)
train_weather['wet_temp_roll_shift14'].fillna(0, inplace = True)
train_weather['wet_temp_roll_shift7'] = train_weather['wet_temp_roll'].shift(7)
train_weather['wet_temp_roll_shift7'].fillna(0, inplace = True)


In [ ]:
test_weather['wet_temp'] = test_weather['PrecipTotal']*test_weather['Tavg']
test_weather['wet_temp_roll'] = test_weather['wet_temp'].rolling(3).mean()
test_weather['wet_temp_roll'].fillna(0, inplace = True)
test_weather['wet_temp_roll_shift14'] = test_weather['wet_temp_roll'].shift(14)
test_weather['wet_temp_roll_shift14'].fillna(0, inplace = True)
test_weather['wet_temp_roll_shift7'] = test_weather['wet_temp_roll'].shift(7)
test_weather['wet_temp_roll_shift7'].fillna(0, inplace = True)

Creating interactive feature of Temp and Precipation and then doing a rolling 3 day mean. Then also shifting that data 1 and 2 weeks. This gives us one feature to account for best mosquito temps and time shifts it for the hatching and larvea time. 

In [ ]:
def date_split(df):
    #breaking week, month, and year into separate columns
    
    df['Week'] = df['Date'].dt.week
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year

    #drop date column
    df.drop(columns = 'Date', inplace = True)

    #get dummies
    return pd.get_dummies(df, columns = ['Week'])
    return pd.get_dummies(df, columns = ['Month'])
    return pd.get_dummies(df, columns = ['Year'])

In [ ]:
#call function
date_split(train_weather)
date_split(test_weather)

In [ ]:
# Plotting out WNV Present by Species
grouped = train.groupby(["Species"])
grouped_percentage = pd.DataFrame()
grouped_percentage["Number of Total Instances"] = train["Species"].value_counts()
grouped_percentage["WnvNotPresent Rate"] = grouped["WnvPresent"].apply(lambda x : x.value_counts()[0]/len(x) )
grouped_percentage["WnvPresent Rate"] = 1 - grouped_percentage["WnvNotPresent Rate"]
grouped_percentage["WnvNotPresent Instances"] = grouped_percentage["Number of Total Instances"] * grouped_percentage["WnvNotPresent Rate"]
grouped_percentage["WnvPresent Instances"] = grouped_percentage["Number of Total Instances"] - grouped_percentage["WnvNotPresent Instances"]

#grouped_percentage

plt.figure(figsize=(15,7))
plt.bar(grouped_percentage.index ,grouped_percentage["WnvPresent Rate"])

In [ ]:
train_weather.drop(columns = ['Address','Street', 'AddressNumberAndStreet', 'Latitude', \
                                              'Longitude', 'AddressAccuracy','LatLong'], inplace = True)
test_weather.drop(columns = ['Address','Street', 'AddressNumberAndStreet', 'Latitude', \
                                              'Longitude', 'AddressAccuracy','LatLong'], inplace = True)

Removing some columns that are not needed for modeling. Block get dummies covers a lot of this data.

In [ ]:
# Train-Train-Split on Data Set

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
X = train_weather.drop('WnvPresent', axis =1)
y = train_weather['WnvPresent']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)


In [ ]:
traindata = X_train.merge(pd.DataFrame(y_train), how = 'left', right_index = True, left_index = True)
train_majority = traindata[traindata['WnvPresent'] == 0]
train_minority = traindata[traindata['WnvPresent'] == 1]
train_minority_upsampled = resample(train_minority, 
                                     replace = True, 
                                     n_samples = train_majority.shape[0],
                                     random_state = 42)

train_data_upsampled = pd.concat([train_majority, train_minority_upsampled])
X_train = train_data_upsampled.drop(columns = 'WnvPresent')
y_train = train_data_upsampled['WnvPresent']

In [ ]:
def drop_columns(df1, df2):
    #drop columns in either test/train that are not in the other
    
    df1cols = df1.columns.tolist()
    df2cols = df2.columns.tolist()
    
    notindf1cols = []
    notindf2cols = []
    
    for col in df1cols:
        if col not in df2cols:
            notindf2cols.append(col)
    
    for col in df2cols:
        if col not in df1cols:
            notindf1cols.append(col)
            
    df1.drop(columns = notindf2cols, inplace = True)
    df2.drop(columns = notindf1cols, inplace = True)

In [ ]:
drop_columns(test_weather, train_weather)


Making sure our Train and Test data is aligned. 

In [ ]:
#checking Class Balance
y_train.value_counts()

In [ ]:
# RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# rf = RandomForestClassifier()

# rf_pipe = Pipeline([
#     ('ss', ss),
#     ('rf', rf)
# ])

# params = {'rf__n_estimators' : [150,200,250,300],
#           'rf__max_depth' : [None, 2, 3, 4, 5]}

# rf_gs = GridSearchCV(rf_pipe, param_grid=params, cv=5, scoring='roc_auc',n_jobs= 5)
# rf_gs.fit(X_train, y_train)

# best_rf_gs = rf_gs.best_estimator_

# rf_gs_train = best_rf_gs.score(X_train, y_train)
# rf_gs_test = best_rf_gs.score(X_test, y_test)

# print(best_rf_gs)
# print(rf_gs_train)
# print(rf_gs_test)
# print(rf_gsbest_params)

Used above GS to determine best params to run on Random Forest below

In [ ]:
rf_gs.best_params_

In [ ]:
rf = RandomForestClassifier(n_estimators= 100, n_jobs = 5)
rf.fit(X_train,y_train)
rf.score(X_test, y_test)


In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')



test_names = X.columns.tolist()
rf_importances = pd.DataFrame(sorted(zip(test_names, rf.feature_importances_), reverse = True), columns = ['Variable', 'Importance']).set_index('Variable')
rf_importances.sort_values(by = 'Importance', ascending = False).iloc[:20,:].plot(kind = 'bar')
plt.title('Random Forest Feature Importances')

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

y_pred_proba = rf.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8,8))
plt.plot(fpr,tpr,lw = 3, color='blue', label='ROC Curve %.5f' % auc)
plt.plot([0,1], [0,1], lw = 3, linestyle ='--', color='grey')
plt.legend(loc=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Random Forest ROC Curve')
plt.savefig('gbc_roc.png')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# Create a heatmap confusion matrix - get predictions 
predictions = rf.predict(X_test)

# Create confusion matrix 
classes = ["No WNV", 'WNV']
cm = confusion_matrix(y_test, predictions)
cm = pd.DataFrame(cm, columns=classes)
cm.index = classes

# Plot matrix on heatmap 
sns.heatmap(cm, annot=True, fmt='g')
plt.title('RF Model');


In [ ]:
# ADABoost
from sklearn.ensemble import AdaBoostClassifier

adamodel = AdaBoostClassifier(n_estimators=100) 
ada_scores = cross_val_score(adamodel, X_train, y_train, cv=5)
adamodel.fit(X_train,y_train)
adamodel.score(X_train,y_train)
y_preds = adamodel.predict(X_test)
adamodel.score(X_test,y_test)

In [ ]:
roc_auc_score(y_test,y_preds)

In [ ]:
ada_importances = pd.DataFrame(sorted(zip(test_names, adamodel.feature_importances_), reverse = True), columns = ['Variable', 'Importance']).set_index('Variable')
ada_importances.sort_values(by = 'Importance', ascending = False).iloc[:20,:].plot(kind = 'bar')
plt.title('Ada Boost Feature Importances')

In [ ]:
adapredicts = pd.DataFrame(y_preds, columns = ['predict'])
#adapredicts.columns
adapredicts['predict'].value_counts()

In [ ]:
y_pred_proba = adamodel.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(6,6))
plt.plot(fpr,tpr,lw = 3, color='blue', label='ROC Curve %.5f' % auc)
plt.plot([0,1], [0,1], lw = 3, linestyle ='--', color='grey')
plt.legend(loc=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Ada Boost ROC Curve')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# Create a heatmap confusion matrix - get predictions 
predictions = adamodel.predict(X_test)

# Create confusion matrix 
classes = ["No WNV", 'WNV']
cm = confusion_matrix(y_test, predictions)
cm = pd.DataFrame(cm, columns=classes)
cm.index = classes

# Plot matrix on heatmap 
sns.heatmap(cm, annot=True, fmt='g')
plt.title('ADA Boost');

In [ ]:
# XGBoost Classifier (This takes about 5 min to finish)

gs_params = {
    'max_depth':[1, 2, 3, 4, 5],
    'n_estimators':range(1, 10, 1),
    'learning_rate':np.logspace(-5,0,5),
    'silent' : [False],
    'booster' : ['gbtree', 'gblinear', 'dart'] 
}

xgb_gs = GridSearchCV(XGBClassifier(), gs_params, cv=5, verbose=1, scoring='roc_auc',n_jobs = -1)

xgb_gs = xgb_gs.fit(X_train, y_train)

best_xgb_gs = xgb_gs.best_estimator_

xgb_gs_train = best_xgb_gs.score(X_train, y_train)
xgb_gs_test = best_xgb_gs.score(X_test, y_test)

print(best_xgb_gs)
print(xgb_gs_train)
print(xgb_gs_test)

In [ ]:
# Gridsearch on logistic regression model above
lr_params = {'penalty':['l1', 'l2'], 
             'C': np.logspace(-5, 2, 10)}
gslr = GridSearchCV(LogisticRegression(), param_grid = lr_params)
gslr.fit(X_train, y_train)

# Results 
gslr.best_score_, gslr.best_params_

In [ ]:
gslr.score(X_test,y_test)

In [ ]:
lrpredict = gslr.predict(X_test)
roc_auc_score(y_test, lrpredict)

In [ ]:
coefs = pd.DataFrame(gslr.best_estimator_.coef_[0], index = X.columns, columns = ['coef'])
coefs.sort_values(by='coef', ascending = False, inplace=True)
coefs.head(20).plot(kind = 'barh')
plt.title('Logistic Regression Coefficients')

In [ ]:
# Create a heatmap confusion matrix - get predictions 
predictions = gslr.predict(X_test)

# Create confusion matrix 
classes = ["No WNV", 'WNV']
cm = confusion_matrix(y_test, predictions)
cm = pd.DataFrame(cm, columns=classes)
cm.index = classes

# Plot matrix on heatmap 
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Logistic Regression');

In [ ]:
y_pred_proba = gslr.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(6,6))
plt.plot(fpr,tpr,lw = 3, color='blue', label='ROC Curve %.5f' % auc)
plt.plot([0,1], [0,1], lw = 3, linestyle ='--', color='grey')
plt.legend(loc=4)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Log Reg ROC Curve')
plt.show()

In [ ]:
# Executive Summary of Models

print('GridSearchCV across Random Forest:')
print(f"Best Parameters = {rf_gs.best_params_}")
print(f"Best CV Score = {rf_gs.best_score_}")
print(f"Train Score = {rf_gs_train}")
print(f"Test Score = {rf_gs_test}")
print()
print('GridSearchCV across XGBoost:')
print(f"Best Parameters = {xgb_gs.best_params_}")
print(f"Best CV Score = {xgb_gs.best_score_}")
print(f"Train Score = {xgb_gs_train}")
print(f"Test Score = {xgb_gs_test}")
print()
print('GridSearchCV across BalancedBaggingClassifier:')
print(f"Best Parameters = {bbc_gs.best_params_}")
print(f"Best CV Score = {bbc_gs.best_score_}")
print(f"Train Score = {bbc_train}")
print(f"Test Score = {bbc_test}")



We picked ADAboost as it optimized for False Negatives. 

In [ ]:
predict = adamodel.predict(test_weather)

In [ ]:


test['WnvPresent'] = predict
test[['Id','WnvPresent']].to_csv('submission.csv',index = False)

In [ ]:
test[['Id','WnvPresent']].to_csv('submission.csv',index = False)